# Library

In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

from keras.utils.np_utils import to_categorical
from keras.models import load_model, Model, Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Embedding, SimpleRNN, Dropout

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Preprocessing
### Load Data

In [2]:
df = pd.read_pickle('preprocessed_df.pkl')
df = pd.DataFrame(df[['detokenize','direktor']])
df.head()

,detokenize,direktor
0,the desperate hour lakewood salah cerita suara...,1
1,edisi males review singkat tonton libur dp des...,0
2,plot utama orang deserter pursuit buru wamil j...,0
3,film hereditary horror thrill midsommar gatau ...,0
4,batman manusiawi tarung nya sadis scene pursui...,0


### Vectorize

In [3]:
df['sent_len'] = df['detokenize'].apply(lambda x: len(x.split(" ")))
max_sequence_length = np.round(df['sent_len'].mean() + 2 * df['sent_len'].std()).astype(int)
max_sequence_length

32

In [4]:
MAX_NB_WORDS = 1e5
le = LabelEncoder()

reviews = df["detokenize"]
label = le.fit_transform(df['direktor'])
label = np.asarray(to_categorical(label))

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(reviews)
review_seq = np.array(tokenizer.texts_to_sequences(reviews), dtype=object)
review_seq = pad_sequences(review_seq, padding='post', maxlen=max_sequence_length)

word_index = tokenizer.word_index
print(f'Dictionary Size\t: {len(word_index)}')
print(f'Example Label\t:\n{label[:5]}')

Dictionary Size	: 17718
Example Label	:
[[0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


# Variables

In [5]:
embedding_dim = 300
vocab_size = min(MAX_NB_WORDS, len(word_index) + 1)
test_size_split = 0.1

# Split Data

In [6]:
x_train, x_test, y_train, y_test = (
    train_test_split(review_seq, 
                     label, 
                     test_size=test_size_split, 
                     stratify = label, 
                     random_state=42)
)

x_train = tf.constant(x_train)
x_test = tf.constant(x_test)

y_train = tf.constant(y_train)
y_test = tf.constant(y_test)

print(f'Train\t| X shape: {x_train.shape}\tY shape: {y_train.shape}')
print(f'Test\t| X shape: {x_test.shape}\tY shape: {y_test.shape}')

Train	| X shape: (15522, 32)	Y shape: (15522, 3)
Test	| X shape: (1725, 32)	Y shape: (1725, 3)


# Model RNN

In [7]:
def RNN(x_train_c, y_train_c, x_test_c, y_test_c):
    rnn = Sequential()
    rnn.add(Embedding(vocab_size, 
                      embedding_dim, 
                      input_length=max_sequence_length,
                      trainable=False))
    rnn.add(SimpleRNN(units=256, activation='relu', recurrent_dropout=0.2, return_sequences=True))
    rnn.add(Dropout(0.2))
    rnn.add(SimpleRNN(units=128, activation='relu', return_sequences=True))
    rnn.add(Dropout(0.2))
    rnn.add(SimpleRNN(units=64, activation='relu'))
    rnn.add(Dropout(0.2))
    rnn.add(Dense(units=3, activation='softmax'))
    
    rnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    early_stopping = EarlyStopping(monitor='val_loss', mode = 'min', verbose = 1, patience=3, min_delta=0.0001)
    model_checkpoint = ModelCheckpoint('rnn_baseline_direktor.h5', monitor = 'val_accuracy', mode = 'max', verbose = 1, save_best_only = True)
    
    history = rnn.fit(x_train_c, y_train_c, epochs= 10, batch_size=128, 
                      validation_data=(x_test_c, y_test_c), verbose = 1,
                      callbacks=[early_stopping, model_checkpoint])
    
    model = load_model('rnn_baseline_direktor.h5')
    y_pred = model.predict(x_test_c)
    y_pred_cat = y_pred.argmax(axis=1)
    y_test_cat = np.argmax(y_test_c, axis=1)
   
    cm = confusion_matrix(y_test_cat, y_pred_cat)
    classreport = classification_report(y_test_cat, y_pred_cat)
    f1 = f1_score(y_test_cat, y_pred_cat,  average="macro")
    recall = recall_score(y_test_cat, y_pred_cat, average="macro")
    precision = precision_score(y_test_cat, y_pred_cat, average="macro")
    accuracy = accuracy_score(y_test_cat, y_pred_cat)

    print(classreport)
    print("F1 Score : ", f1)
    print("Precision : ", precision)
    print("Recall : ", recall)
    print("Accuracy : ", accuracy)

    return [f1, precision, recall, accuracy, cm]

# Running

In [8]:
hasil = []

for i in range(1, 6):
    hasil.append([str(i)] + RNN(x_train, y_train, x_test, y_test))

Epoch 1/10
122/122 [==============================] - ETA: 0s - loss: 0.5271 - accuracy: 0.8477
Epoch 1: val_accuracy improved from -inf to 0.85507, saving model to rnn_baseline_direktor.h5
122/122 [==============================] - 30s 165ms/step - loss: 0.5271 - accuracy: 0.8477 - val_loss: 0.4918 - val_accuracy: 0.8551
Epoch 2/10
122/122 [==============================] - ETA: 0s - loss: 0.4951 - accuracy: 0.8554
Epoch 2: val_accuracy did not improve from 0.85507
122/122 [==============================] - 18s 150ms/step - loss: 0.4951 - accuracy: 0.8554 - val_loss: 0.5143 - val_accuracy: 0.8551
Epoch 3/10
122/122 [==============================] - ETA: 0s - loss: 0.4920 - accuracy: 0.8554
Epoch 3: val_accuracy did not improve from 0.85507
122/122 [==============================] - 19s 152ms/step - loss: 0.4920 - accuracy: 0.8554 - val_loss: 0.4833 - val_accuracy: 0.8551
Epoch 4/10
122/122 [==============================] - ETA: 0s - loss: 0.4841 - accuracy: 0.8554
Epoch 4: val_accur

C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anac

Epoch 1/10
122/122 [==============================] - ETA: 0s - loss: 0.5256 - accuracy: 0.8496
Epoch 1: val_accuracy improved from -inf to 0.85507, saving model to rnn_baseline_direktor.h5
122/122 [==============================] - 33s 187ms/step - loss: 0.5256 - accuracy: 0.8496 - val_loss: 0.5005 - val_accuracy: 0.8551
Epoch 2/10
122/122 [==============================] - ETA: 0s - loss: 0.4981 - accuracy: 0.8554
Epoch 2: val_accuracy did not improve from 0.85507
122/122 [==============================] - 18s 149ms/step - loss: 0.4981 - accuracy: 0.8554 - val_loss: 0.4867 - val_accuracy: 0.8551
Epoch 3/10
122/122 [==============================] - ETA: 0s - loss: 0.4906 - accuracy: 0.8554
Epoch 3: val_accuracy did not improve from 0.85507
122/122 [==============================] - 17s 135ms/step - loss: 0.4906 - accuracy: 0.8554 - val_loss: 0.4843 - val_accuracy: 0.8551
Epoch 4/10
122/122 [==============================] - ETA: 0s - loss: 0.4872 - accuracy: 0.8554
Epoch 4: val_accur

C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anac

Epoch 1/10
122/122 [==============================] - ETA: 0s - loss: 0.5260 - accuracy: 0.8517
Epoch 1: val_accuracy improved from -inf to 0.85507, saving model to rnn_baseline_direktor.h5
122/122 [==============================] - 34s 205ms/step - loss: 0.5260 - accuracy: 0.8517 - val_loss: 0.4852 - val_accuracy: 0.8551
Epoch 2/10
122/122 [==============================] - ETA: 0s - loss: 0.4952 - accuracy: 0.8554
Epoch 2: val_accuracy did not improve from 0.85507
122/122 [==============================] - 22s 183ms/step - loss: 0.4952 - accuracy: 0.8554 - val_loss: 0.4845 - val_accuracy: 0.8551
Epoch 3/10
122/122 [==============================] - ETA: 0s - loss: 0.4940 - accuracy: 0.8554
Epoch 3: val_accuracy did not improve from 0.85507
122/122 [==============================] - 27s 219ms/step - loss: 0.4940 - accuracy: 0.8554 - val_loss: 0.4839 - val_accuracy: 0.8551
Epoch 4/10
122/122 [==============================] - ETA: 0s - loss: 0.4842 - accuracy: 0.8554
Epoch 4: val_accur

C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anac

Epoch 1/10
122/122 [==============================] - ETA: 0s - loss: 0.5279 - accuracy: 0.8521
Epoch 1: val_accuracy improved from -inf to 0.85507, saving model to rnn_baseline_direktor.h5
122/122 [==============================] - 31s 182ms/step - loss: 0.5279 - accuracy: 0.8521 - val_loss: 0.5023 - val_accuracy: 0.8551
Epoch 2/10
122/122 [==============================] - ETA: 0s - loss: 0.4966 - accuracy: 0.8554
Epoch 2: val_accuracy did not improve from 0.85507
122/122 [==============================] - 19s 152ms/step - loss: 0.4966 - accuracy: 0.8554 - val_loss: 0.5272 - val_accuracy: 0.8551
Epoch 3/10
122/122 [==============================] - ETA: 0s - loss: 0.4911 - accuracy: 0.8554
Epoch 3: val_accuracy did not improve from 0.85507
122/122 [==============================] - 20s 163ms/step - loss: 0.4911 - accuracy: 0.8554 - val_loss: 0.4797 - val_accuracy: 0.8551
Epoch 4/10
122/122 [==============================] - ETA: 0s - loss: 0.4851 - accuracy: 0.8554
Epoch 4: val_accur

C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anac

Epoch 1/10
122/122 [==============================] - ETA: 0s - loss: 0.5246 - accuracy: 0.8512
Epoch 1: val_accuracy improved from -inf to 0.85507, saving model to rnn_baseline_direktor.h5
122/122 [==============================] - 33s 197ms/step - loss: 0.5246 - accuracy: 0.8512 - val_loss: 0.4927 - val_accuracy: 0.8551
Epoch 2/10
122/122 [==============================] - ETA: 0s - loss: 0.5029 - accuracy: 0.8554
Epoch 2: val_accuracy did not improve from 0.85507
122/122 [==============================] - 21s 168ms/step - loss: 0.5029 - accuracy: 0.8554 - val_loss: 0.4854 - val_accuracy: 0.8551
Epoch 3/10
122/122 [==============================] - ETA: 0s - loss: 0.4989 - accuracy: 0.8554
Epoch 3: val_accuracy did not improve from 0.85507
122/122 [==============================] - 20s 162ms/step - loss: 0.4989 - accuracy: 0.8554 - val_loss: 0.4897 - val_accuracy: 0.8551
Epoch 4/10
122/122 [==============================] - ETA: 0s - loss: 0.5013 - accuracy: 0.8554
Epoch 4: val_accur

C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alienware 15R2\anac

# Save Output

In [9]:
df = pd.DataFrame(hasil, columns =['iterasi', 'f1', 'precision', 'recall', 'accuracy', 'cm'])
#df.to_excel('rnn_baseline_direktor.xlsx')
df.head()

,iterasi,f1,precision,recall,accuracy,cm
0,1,0.358658,0.530348,0.360353,0.862029,"[[0, 57, 2], [0, 1471, 4], [0, 175, 16]]"
1,2,0.448466,0.525668,0.426771,0.876522,"[[0, 53, 6], [0, 1456, 19], [0, 135, 56]]"
2,3,0.461584,0.522107,0.440506,0.880580,"[[0, 48, 11], [0, 1455, 20], [0, 127, 64]]"
3,4,0.307292,0.285024,0.333333,0.855072,"[[0, 59, 0], [0, 1475, 0], [0, 191, 0]]"
4,5,0.465863,0.533119,0.442930,0.882899,"[[0, 49, 10], [0, 1458, 17], [0, 126, 65]]"


In [12]:
df['f1'].mean()

0.408372326133002

In [13]:
df['accuracy'].mean()

0.8714202898550724